In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
#Objetos que dan formato al dataframe
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.sql.types import Row
from pyspark.sql import SQLContext

In [2]:
sc = SparkContext(master="local", appName="tranfYAcc")

In [26]:
#Se llama
sqlContext = SQLContext(sc)

A:\Instal_python\lib\site-packages\pyspark\sql\context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
rddl = sc.parallelize([1,2,3])
type(rddl)

pyspark.rdd.RDD

In [4]:
#Visualizar Datos
rddl.collect()

[1, 2, 3]

In [5]:
var = "C:/Users/Caroline/Videos/curso-apache-spark-platzi/files/"

prubRDD = sc.textFile(var+"paises.csv").map(lambda line : line.split(","))

In [6]:
#Ver datos
prubRDD.take(15)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR'],
 ['10', 'Alcyon-6', 'FRA'],
 ['11', 'Alcyon-7', 'FRA'],
 ['12', 'Aldebaran', 'ITA'],
 ['13', 'Aldebaran II', 'ITA'],
 ['14', 'Aletta', 'IRL']]

In [7]:
#CONTAR
#Seleccionar columna e ingresarlo en una columna
#Se usa distinct para evitar duplicados
prubRDD.map(lambda x: (x[2])).distinct().count()

231

In [8]:
#GROUP BY
#Se invierte el valor, para que agrupe por equipo y no por llave valor
#Se debe agrupar con llave
#Take: Ver pocos datos de todo el conjunto
prubRDD.map(lambda x : (x[2],x[1])).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [9]:
#Filtrar valores
#Collect --> Consulta total y puede sobrecargar los clústeres
#Equipos México
eqmex = prubRDD.filter(lambda l : "MEX" in l)
eqmex.collect()

[['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['193', 'Chamukina', 'MEX'],
 ['656', 'Mexico', 'MEX'],
 ['657', 'Mexico-1', 'MEX'],
 ['658', 'Mexico-2', 'MEX'],
 ['700', 'Nausikaa 4', 'MEX'],
 ['1053', 'Tlaloc', 'MEX'],
 ['1168', 'Xolotl', 'MEX']]

In [10]:
#count --> Hace el conteo total de todos los datos del archivo
#countApprox --> Cuenta la cantidad de registros recuperados en milisegundos
prubRDD.countApprox(20)

1185

In [11]:
#Importar datos
deporrRDD = sc.textFile(var+"deportista.csv").map(lambda line : line.split(","))
deporrRDD2 = sc.textFile(var+"deportista2.csv").map(lambda line : line.split(","))

In [12]:
#Union de los datos
deporrRDD  = deporrRDD.union(deporrRDD2)

In [ ]:
#Top
prubRDD.top(2)

In [ ]:
deporrRDD.top(2)

In [13]:
#Hacer un Join
#Seleccionar campo clave del primer RDD
#Hacer Join con campo clave del segundo RDD
deporrRDD.map(lambda l: [l[-1], l[:-1]]).join(prubRDD.map(lambda x: [x[0],x[2]])).take(5)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN'))]

In [14]:
#Hacer un Join
#Seleccionar campo clave del primer RDD
#Hacer Join con campo clave del segundo RDD
#takeSample --> Muetra aleatoria
deporrRDD.map(lambda l: [l[-1], l[:-1]]).join(prubRDD.map(lambda x: [x[0],x[2]])).takeSample(False,6,25)

[('362', (['131505', 'Steven Woodburn', '1', '24', '185', '90'], 'FRA')),
 ('967', (['13626', 'Jill Brresen', '2', '22', '170', '57'], 'RSA')),
 ('482', (['44299', 'Gumundur Gumundsson', '1', '23', '174', '77'], 'ISL')),
 ('970', (['68062', 'Lee MinHui', '2', '28', '174', '65'], 'KOR')),
 ('794', (['92442', 'Luis Paz Zoldan', '1', '19', '187', '82'], 'PER')),
 ('413', (['26822', 'Jared Mark Deacon', '1', '24', '185', '77'], 'GBR'))]

In [15]:
result = sc.textFile(var+"resultados.csv").map(lambda line : line.split(","))

In [16]:
#Eliminar datos
resultG = result.filter(lambda l: "NA" not in l[1])

In [ ]:
resultG.take(2)

In [ ]:
deporrRDD.top(2)

In [17]:
#Se realiza el Join de datos
deporPais = deporrRDD.map(lambda l : [l[-1], l[:-1]]).join(prubRDD.map(lambda x: [x[0], x[2]]))

In [ ]:
deporPais.top(2)

In [18]:
deporPais.join(resultG).take(2)

[('74',
  ((['65', 'Patimat Abakarova', '2', '21', '165', '49'], 'AZE'), 'Gold')),
 ('74', ((['129', 'Ruslan Abbasov', '1', '22', '181', '74'], 'AZE'), 'Gold'))]

In [19]:
#Función para validar el encabezado
#Iter: Devuelve valor a valor lo que se está procesando
def deletHeader(indice, interador):
    return iter(list(interador)[1:])

In [20]:
#Eliminar el encabezado
deporrRDD = deporrRDD.mapPartitionsWithIndex(deletHeader)

In [21]:
deporrRDD.take(4)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [22]:
#Antes de Tramsformar los datos del RDD, se debe transformar el tipo de dato
deporrRDD = deporrRDD.map(lambda l: (
int(l[0]),
l[1],    
int(l[2]),
int(l[3]),
int(l[4]),
float(l[5]),
int(l[6])
))

In [24]:
#Se crear el Schema
depor = StructType([
    StructField("deportista_id", IntegerType(), False),
    StructField("nombre", StringType(), False),
    StructField("genero", IntegerType(), False),
    StructField("edad", IntegerType(), False),
    StructField("altura", IntegerType(), False),
    StructField("peso", FloatType(), False),
    StructField("equipo_id", IntegerType(), False)
])

In [28]:
#Aplicar el esquema al RDD
sqlContext.createDataFrame(deporrRDD, depor)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows

